# Mixed View

## Set correct directory

In [1]:
cd ..

/bptk-py


## Import libraries

In [2]:
import pandas as pd
import time
import pickle
from elasticsearch import Elasticsearch
import os.path

## Batch View

In [3]:
# simple query just unpickles the data
data_file_object = open ("csv/data.pickle",'rb')
data=pickle.load(data_file_object)
data

{'time': 14400,
 '1_car': {'time': 14400,
  'timestamp': datetime.datetime(2019, 11, 19, 20, 43, 36, 26875),
  'id': 1,
  'sum_revenue': 2870.640051037073,
  'sum_cost': 4173.200163900852,
  'profit': -1302.560112863779,
  'avg_sum_revenue': 0.19935000354424118,
  'avg_sum_cost': 0.28980556693755916,
  'avg_profit': -0.09045556339331799},
 '2_car': {'time': 14400,
  'timestamp': datetime.datetime(2019, 11, 19, 20, 43, 36, 487132),
  'id': 2,
  'sum_revenue': 2929.520051717758,
  'sum_cost': 4174.100163966417,
  'profit': -1244.5801122486591,
  'avg_sum_revenue': 0.20343889248039987,
  'avg_sum_cost': 0.2898680669421123,
  'avg_profit': -0.08642917446171244},
 '3_car': {'time': 14400,
  'timestamp': datetime.datetime(2019, 11, 19, 20, 43, 37, 518265),
  'id': 3,
  'sum_revenue': 2878.2000516057014,
  'sum_cost': 4176.600164070725,
  'profit': -1298.400112465024,
  'avg_sum_revenue': 0.19987500358372925,
  'avg_sum_cost': 0.29004167806046705,
  'avg_profit': -0.09016667447673778}}

In [36]:
time_file_object = open("csv/sim_time.pickle",'rb')  
current_time = pickle.load(time_file_object)
current_time

21600

## Real Time View

## Mixed Query Using Batch and Real Time Views

In [37]:
es = Elasticsearch([{'host': 'es_node1', 'port': 9200}])

query_time = 21533
# find smallest multiple of 1440 less than query time

batch_time = 1440*(query_time//1440)
rt_time = query_time-batch_time

batch_result = es.search(index="1_car_batch",body={"size": 0,
                "aggs" : {
                    "revenue" : {
                        "date_range": {
                            "field": "time",
                            "ranges": [
                                { "from": batch_time,
                                  "to": batch_time+1 } 
                            ]
                        },
                        "aggs": {
                            "total_revenue": {
                              "sum": {
                                "field": "sum_revenue"
                              }
                            }
                        }
                    }
                }})

rt_result = es.search(index="1_car",body={"size": 0,
                "aggs" : {
                    "revenue" : {
                        "date_range": {
                            "field": "time",
                            "ranges": [
                                { "from": batch_time+1,
                                  "to": batch_time+rt_time+1 } 
                            ]
                        },
                        "aggs": {
                            "total_revenue": {
                              "sum": {
                                "field": "revenue"
                              }
                            }
                        }
                    }
                }})

full_scan_result = es.search(index="1_car",body={"size": 0,
                "aggs" : {
                    "revenue" : {
                        "date_range": {
                            "field": "time",
                            "ranges": [
                                { "from": 0,
                                  "to": batch_time+rt_time+1 } 
                            ]
                        },
                        "aggs": {
                            "total_revenue": {
                              "sum": {
                                "field": "revenue"
                              }
                            }
                        }
                    }
                }})

car_result = es.search(index="1_car",body={"size": 0,
                "aggs" : {
                    "revenue" : {
                        "date_range": {
                            "field": "time",
                            "ranges": [
                                { "from": batch_time+rt_time,
                                  "to": batch_time+rt_time+1 } 
                            ]
                        },
                        "aggs": {
                            "total_revenue": {
                              "sum": {
                                "field": "total_revenue"
                              }
                            }
                        }
                    }
                }})

difference = full_scan_result["aggregations"]["revenue"]["buckets"][0]["total_revenue"]["value"] - batch_result["aggregations"]["revenue"]["buckets"][0]["total_revenue"]["value"]- rt_result["aggregations"]["revenue"]["buckets"][0]["total_revenue"]["value"]

In [38]:
batch_result

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 15, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'revenue': {'buckets': [{'key': '20160.0-20161.0',
     'from': 20160.0,
     'to': 20161.0,
     'doc_count': 1,
     'total_revenue': {'value': 3000.679931640625}}]}}}

In [39]:
rt_result

{'took': 274,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'revenue': {'buckets': [{'key': '20161.0-21534.0',
     'from': 20161.0,
     'to': 21534.0,
     'doc_count': 1373,
     'total_revenue': {'value': 0.0}}]}}}

In [40]:
full_scan_result

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'revenue': {'buckets': [{'key': '0.0-21534.0',
     'from': 0.0,
     'to': 21534.0,
     'doc_count': 21533,
     'total_revenue': {'value': 3000.6800533235073}}]}}}

In [41]:
difference

0.00012168288230895996

In [42]:
car_result

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'revenue': {'buckets': [{'key': '21533.0-21534.0',
     'from': 21533.0,
     'to': 21534.0,
     'doc_count': 1,
     'total_revenue': {'value': 3000.679931640625}}]}}}